Основная проблема - необходимо заполнить таблицу результатов продаж на +7 и +30 дней со дня поставки товара. Поэтому необходимо запускать скрипты для каждой поставки-материала отдельно

In [9]:
import psycopg2
import pandas as pd
import datetime as dt
import tqdm.notebook as tqdm

In [10]:
# Создаем реквизиты для входа в GP
DBNAME = 'XXXX'
USER ='XXXX'
PASSWORD ='XXXX'
HOST ='XXXX'
PORT = 1111

In [11]:
# Сегодня
today=dt.datetime.today()
# Сегодня - 7 дней
today_7=dt.datetime.today() - dt.timedelta(days=8)
# Сегодня - 30 дней
today_30=dt.datetime.today() - dt.timedelta(days=31)

In [12]:
def insert_new_data():
    '''
    Функция внесения данных
    '''
    # Импорт поставок от товародвижения
    postavki = pd.read_excel('postavki_redistr.xlsx')
    # Перевод df into list
    postavki=postavki['Поставки'].to_list()
    # Добавляем 0 впереди, если нет
    postavki = [str(item).zfill(10) for item in postavki]
    # превращаем в формат (1111), (2222)... для insert
    postavki = "('"+"'),('".join(map(str, postavki))+"')"
    
    sql_post_truncate = "TRUNCATE scm.postavki_temp"
    
    sql_post = "INSERT INTO scm.postavki_temp(postavka) VALUES {}".format(postavki)
    
    sql_delete_existing_post = '''
    DELETE FROM scm.postavki_temp as rt
    WHERE rt.postavka in (SELECT DISTINCT postavka FROM scm.postavki)
    '''
    
    sql_add_post_to_upd = '''
    INSERT INTO scm.postavki_temp
    SELECT postavka 
    FROM scm.postavki
    WHERE 
        (date_in is null
         and
         date_out > '{}')
         or
        (date_out is null)
    '''.format(today_30.strftime('%Y%m%d'))
    
    sql_delete_to_upd = '''
    DELETE FROM scm.postavki
    WHERE 
        (date_in is null
         and
         date_out > '{}')
         or
        (date_out is null)
    '''.format(today_30.strftime('%Y%m%d'))
    
    sql_in_sum = '''
    SELECT
         max(zpstngdat) as date_provodki
        ,coalesce(zmovplant, zvendor) as from_store -- coalesce нужен при 101 движении
        ,zplant as to_store  
        ,zmaterial as sku
        ,zrefdocno as postavka
        ,sum(zcpquabu) as pcs -- проверить надо
        ,zebeln as zakaz_na_perem
        ,sum(zastcnvat) as rub_no_nds
        ,sum(zastcvat) as rub_with_nds
    FROM dbo.account_book
     WHERE 
     zoi_myear = date_part('year', CURRENT_DATE)::text
     AND zrefdocno in (SELECT postavka FROM scm.postavki_temp)
     AND zstr_loc is not null -- только движения без ожидаемого прихода
     AND zmovetype in ('101', '102')

    GROUP BY
         zplant
        ,coalesce(zmovplant, zvendor)
        ,zmaterial
        ,zrefdocno
        ,zebeln
    '''
    
    sql_out_sum = '''
    SELECT
         min(zpstngdat) as date_provodki
        ,zplant as from_store
        ,coalesce(zmovplant, zvendor) as to_store -- coalesce нужен при 101 движении
        ,zmaterial as sku
        ,zrefdocno as postavka
        ,sum(zcpquabu) as pcs -- проверить надо
        ,zebeln as zakaz_na_perem
        ,sum(zastcnvat) as rub_no_nds
        ,sum(zastcvat) as rub_with_nds
    FROM dbo.account_book
     WHERE 
     zoi_myear = date_part('year', CURRENT_DATE)::text
     AND zrefdocno in (SELECT postavka FROM scm.postavki_temp)
     AND zstr_loc is not null -- только движения без ожидаемого прихода
     AND zmovetype in ('641', '642')

    GROUP BY
         zplant
        ,coalesce(zmovplant, zvendor)
        ,zmaterial
        ,zrefdocno
        ,zebeln
    '''
    
    sql_upd_moves = '''
    INSERT INTO scm.postavki
    SELECT 
         out_.date_provodki as date_out
        ,out_.from_store
        ,out_.to_store -- coalesce нужен при 101 движении
        ,out_.sku
        ,out_.postavka
        ,out_.pcs as pcs_send
        ,out_.zakaz_na_perem
        ,out_.rub_no_nds as rub_no_nds_send
        ,out_.rub_with_nds as rub_with_nds_send
        ,in_.date_provodki as date_in
        ,in_.pcs as pcs_in

    FROM ({}) as out_
    LEFT JOIN ({}) as in_ ON out_.postavka = in_.postavka AND out_.sku = in_.sku
    '''.format(sql_out_sum, sql_in_sum)
    
    conn = psycopg2.connect(dbname=DBNAME, user=USER, 
                            password=PASSWORD, host=HOST, port=PORT)
    cursor = conn.cursor()
    cursor.execute(sql_post_truncate)
    print('Временные поставки удалены')
    conn.commit()
    cursor.execute(sql_post)
    print('Временные поставки вставлены')
    conn.commit()
    cursor.execute(sql_delete_existing_post)
    print('Удалены поставки, которые уже существуют')
    conn.commit()
    cursor.execute(sql_add_post_to_upd)
    conn.commit()
    print('Добавлены существующие непринятые поставки')
    cursor.execute(sql_delete_to_upd)
    conn.commit()
    print('Удалены существующие непринятые поставки')
    cursor.execute(sql_upd_moves)
    print('Данные загружены')
    conn.commit()
    conn.close()
    print('Загрузка новых данных завершена')
    print('It\'s party time!')

In [13]:
def update_sales(days=7):
    # days=7 кол-во дней назад. Может быть только 7 или 30 !!!!!!!!
    
    today_prev = dt.datetime.today() - dt.timedelta(days=days+1)
    
    # Переменная с sql. Выкачивает только незаполненные продажи, больше days дней назад
    sql_what_update_sales = '''
    SELECT 
         date_in
        ,to_store 
        ,sku
    FROM scm.postavki
    WHERE 
        sales_pcs_{} is null
        AND date_in <= '{}'
    '''.format(days, today_prev.strftime('%Y%m%d'))
    # запуск подключения
    conn = psycopg2.connect(dbname=DBNAME, user=USER, 
                        password=PASSWORD, host=HOST, port=PORT)
    cursor = conn.cursor()
    data_to_update = pd.read_sql_query(sql_what_update_sales, conn)
    conn.close()
    stores=data_to_update['to_store'].unique()
    
    for store in tqdm.tqdm(stores):
        dates_list=data_to_update['date_in'][data_to_update['to_store']==store].unique()
        for date_start in dates_list:
            date_end = date_start + dt.timedelta(days=days+1)
            skus=data_to_update['sku'][(data_to_update['to_store']==store) & (data_to_update['date_in']==date_start)].unique()
            sql_update = '''
            UPDATE scm.postavki
            SET
                 sales_pcs_{} = sales.pcs
                ,sales_rub_vat_in_{} = sales.rub_with_vat
            FROM 
            (
                SELECT
                     s.zplant as store
                    ,s.zmaterial as sku
                    ,sum(s.zrs_rlq) as pcs
                    ,sum(s.zrs_sat) as rub_with_vat
                FROM dbo.azrs_172 as s
                LEFT JOIN dm.d_material as mat ON mat.material_id = s.zmaterial
                WHERE 
                    (s.zrs_hide = 'A' or s.zrs_hide is null) -- скрывать строку или нет. Видимо, черная бухгалтерия
                    AND mat.normal_cbu_flag = True -- zcbu not in (11,20,21,22,23,24)
                    AND s.zrs_ttc not in ('7102','7101') -- удаляем авансовые чеки '7102','7101'
                    AND s.zplant = '{}'
                    AND s.zcalday between '{}' and '{}'
                    AND s.zmaterial in ({})
                GROUP BY
                     s.zplant
                    ,s.zmaterial
            ) as sales
            WHERE
                scm.postavki.to_store = sales.store
                AND scm.postavki.date_in = '{}'
                AND scm.postavki.sku = sales.sku
            '''.format(days, days, store, date_start.strftime('%Y%m%d'), date_end.strftime('%Y%m%d'), ','.join(map(str, skus)), date_start.strftime('%Y%m%d'))
            conn = psycopg2.connect(dbname=DBNAME, user=USER, 
                            password=PASSWORD, host=HOST, port=PORT)
            cursor = conn.cursor()
            cursor.execute(sql_update)
            updated_rows = cursor.rowcount
            conn.commit()
            conn.close()

In [ ]:
insert_new_data()

In [ ]:
update_sales(days=7)

In [ ]:
update_sales(days=30)